# Refinement of labels

# Step1. Start edit_labels
(2022/06/02) Currently, Jupyter cannot run edit_labels with a graph as sub-process. Please run from VS code or command prompt.<BR>

Specify each path in input.csv file.
- **inferred path:** folder path that contains inferred_video and inferred_h5
- **inferred_video:**
- **inferred_h5:** h5 file that contains inferred coordinate from the inferred_video
- **training_path:** DLC training-datasets folder that contains labeled_h5 and labeled_for_train_pickle
- **labeled_h5**: Usually, CollectedData_(scorer).h5
- **labeled_for_train_pickle**: Usually, Documentation_data-(project)*.pickle
    
Once the refinement is done, the script generates (date)-(time)-extracted folder that contains extracted.h5 and .csv and png image files.
    

In [2]:
import edit_labels as el
import os


if __name__ == '__main__':

    # input data
    
    input_path = r'W:\wataru\dlc_data\homecage_test01-wi-2022-06-03\labeled-data\input.csv'
    
    if os.path.exists(input_path):
        inferred_video, inferred_h5, labeled_h5, labeled_for_train_pickle = el.read_input(
            input_path, 1)
    else:
        ############################
        # example data
        # inferred video
        inferred_video = r'input_data\rpicam-01_1806_20210722_212134.mp4'
        # inferred result h5
        inferred_h5 = r'input_data\rpicam-01_1806_20210722_212134DLC_dlcrnetms5_homecage_test01May17shuffle1_200000_el.h5'
        # labeled data for training
        labeled_h5 = r'input_data\CollectedData_DJ.h5'
        # information which frame is used for training or testing
        labeled_for_train_pickle = r'input_data\Documentation_data-homecage_test01_95shuffle1.pickle'

    # video display magnification factor
    mag_factor = 1
    # set window size and position. win_y_len_axis is only for x-axis window.
    window_geo = {'win_x_len': 1000, 'win_y_len': 100, 'win_y_len_axis': 30,
                  'win_x_origin': 0, 'win_y_origin': 0}

    el.start(inferred_video=inferred_video, inferred_h5=inferred_h5,
             labeled_h5=labeled_h5, labeled_for_train_pickle=labeled_for_train_pickle,
             mag_factor=mag_factor, window_geo=window_geo, plot_type='')

video resolution: (960, 1280, 3)
total frame number: 18000
Reconstructed total rows are : 205
Reading _rpicam-01_1806_20210722_212134_track_freeze.csv
Writing _rpicam-01_1806_20210722_212134_track_freeze.csv
Writing _rpicam-01_1806_20210722_212134_freeze.csv


# Step2. Merge labeled h5 files for re-training
## This method works when the added labels come from the original video

#### 1. Process output from edit_labels.
**extracted.h5** file is in the folder, like "20220602-210804-extracted".

In [9]:
import os
import pandas as pd

labeled_data_path = r'W:\wataru\dlc_data\homecage_test01-wi-2022-06-03\labeled-data\rpicam-01_1806_20210722_212134'

# read additional labeled h5 file
added_h5_path = '2_extracted/extracted.h5'
df_add = pd.read_hdf(os.path.join(labeled_data_path, added_h5_path))

# drop likelihood
df_add = df_add.drop(['likelihood'], axis=1, level=3)

# adjust multi-index index
a = ["img{:0>5d}.png".format(x) for x in df_add.index]
idx = pd.MultiIndex.from_product([['labeled-data'],['rpicam-01_1806_20210722_212134'],a])
df_add.index = idx

# set multi-index column
_a = df_add.columns.levels[0].str.replace('DLC_dlcrnetms5_homecage_test01May17shuffle1_200000','wi')
df_add.columns = df_add.columns.set_levels(_a, level=0)

df_add

scorer                                                         wi           \
individuals                                                  sub1            
bodyparts                                                   snout            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  918.000  381.000   
                                            img00266.png      NaN      NaN   
                                            img00276.png      NaN      NaN   
                                            img00334.png      NaN      NaN   
                                            img00335.png      NaN      NaN   
...                                                           ...      ...   
                                            img17492.png  281.519  944.115   
                                            img17515.png      NaN      NaN   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  455.757  777.690   

scorer                                                                      \
individuals                                                                  
bodyparts                                                 leftear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  986.000  322.000   
                                            img00266.png  101.000  295.000   
                                            img00276.png   91.000  327.000   
                                            img00334.png  133.000  638.000   
                                            img00335.png  148.000  682.000   
...                                                           ...      ...   
                                            img17492.png  213.000  860.000   
                                            img17515.png  118.000  593.000   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  462.038  860.248   

scorer                                                                      \
individuals                                                                  
bodyparts                                                rightear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  912.000  292.000   
                                            img00266.png  158.000  243.000   
                                            img00276.png  133.000  261.000   
                                            img00334.png      NaN      NaN   
                                            img00335.png      NaN      NaN   
...                                                           ...      ...   
                                            img17492.png      NaN      NaN   
                                            img17515.png  119.000  424.000   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  479.199  801.207   

scorer                                                                      \
individuals                                                                  
bodyparts                                                tailbase            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  934.187  177.223   
                                            img00266.png  168.000  459.000   
                   

#### 2. Read original labeled h5 file
Search **CollectedData_\*.h5** under dlc_data/\*/training-datasets/iteration-*.

In [16]:
# read original labeled h5 file

labeled_h5_path = '1_original_dalton/CollectedData_DJ.h5'
df_org = pd.read_hdf(os.path.join(labeled_data_path, labeled_h5_path))

_a = df_org.columns.levels[0].str.replace('DJ','wi')
df_org.columns = df_add.columns.set_levels(_a, level=0)
df_org

scorer                                                            wi  \
individuals                                                     sub1   
bodyparts                                                      snout   
coords                                                             x   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  917.600984   
                                            img00266.png  851.908829   
                                            img00276.png         NaN   
                                            img00305.png         NaN   
                                            img00334.png         NaN   
...                                                              ...   
                                            img17492.png  283.042717   
                                            img17515.png   84.568546   
                                            img17842.png  148.862996   
                                            img17860.png         NaN   
                                            img17968.png  238.316143   

scorer                                                                \
individuals                                                            
bodyparts                                                              
coords                                                             y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  376.824559   
                                            img00266.png  204.906791   
                                            img00276.png         NaN   
                                            img00305.png         NaN   
                                            img00334.png         NaN   
...                                                              ...   
                                            img17492.png  944.292965   
                                            img17515.png  506.811164   
                                            img17842.png  729.046328   
                                            img17860.png         NaN   
                                            img17968.png  710.876157   

scorer                                                                \
individuals                                                            
bodyparts                                                    leftear   
coords                                                             x   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  973.509202   
                                            img00266.png  899.430814   
                                            img00276.png   90.000000   
                                            img00305.png         NaN   
                                            img00334.png         NaN   
...                                                              ...   
                                            img17492.png  225.736795   
                                            img17515.png  108.329538   
                                            img17842.png         NaN   
                                            img17860.png         NaN   
                                            img17968.png  179.612515   

scorer                                                                \
individuals                                                            
bodyparts                                                              
coords                                                             y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  325.109458   
                                            img00266.png  210.497612   
                                            img00276.png  327.000000   
                                            img00305.png         NaN   
                                            img00334.png         NaN   
...                                                              ...   
                                     

#### 3. Concatenate both and drop duplication

In [17]:
# concatenate
_df = pd.concat([df_add, df_org])

# drop old duplicate labels and sort
df_merged = _df[~_df.index.duplicated(keep='first')].sort_index(level=2)
df_merged

scorer                                                         wi           \
individuals                                                  sub1            
bodyparts                                                   snout            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  918.000  381.000   
                                            img00266.png      NaN      NaN   
                                            img00276.png      NaN      NaN   
                                            img00305.png      NaN      NaN   
                                            img00334.png      NaN      NaN   
...                                                           ...      ...   
                                            img17492.png  281.519  944.115   
                                            img17515.png      NaN      NaN   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  455.757  777.690   

scorer                                                                      \
individuals                                                                  
bodyparts                                                 leftear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  986.000  322.000   
                                            img00266.png  101.000  295.000   
                                            img00276.png   91.000  327.000   
                                            img00305.png      NaN      NaN   
                                            img00334.png  133.000  638.000   
...                                                           ...      ...   
                                            img17492.png  213.000  860.000   
                                            img17515.png  118.000  593.000   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  462.038  860.248   

scorer                                                                      \
individuals                                                                  
bodyparts                                                rightear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  912.000  292.000   
                                            img00266.png  158.000  243.000   
                                            img00276.png  133.000  261.000   
                                            img00305.png      NaN      NaN   
                                            img00334.png      NaN      NaN   
...                                                           ...      ...   
                                            img17492.png      NaN      NaN   
                                            img17515.png  119.000  424.000   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  479.199  801.207   

scorer                                                                      \
individuals                                                                  
bodyparts                                                tailbase            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  934.187  177.223   
                                            img00266.png  168.000  459.000   
                   

#### 4. Save the merged data as CollectedData_\*.h5

In [18]:
merged_h5_path = '2_extracted/CollectedData_DJ.h5'

df_merged.to_hdf(os.path.join(labeled_data_path, merged_h5_path),key='df_output', mode='w')

#### 5. Move the CollectedData_\*.h5 and png image files to dlc_data/\*/labeled-data/(video_name)/

---